# 2. Simple and Multiple Linear Regression using Gradient Descent (stream data)



*Group Members*
1. 1741031: Foram Vadher
2. 1741034: Krinali Shah
3. 1741074: Arpit Patel
4. 1741079: Vaishwi Patel


Importing Libraries

In [0]:
#Loading boston dataset to csv file
import pandas as pd
from sklearn import datasets, metrics 
import numpy as np


Loading Data set

In [2]:
X, y = datasets.load_boston(return_X_y=True)

y=np.reshape(y,newshape=(y.shape[0],1))
boston=np.hstack((X,y))
print("Boston Shape",boston.shape)
print("X Shape:", X.shape, "y Shape:", y.shape)
boston=pd.DataFrame(boston)
boston.to_csv('boston.csv', header='None', index=False) # store data into cvs file

Boston Shape (506, 14)
X Shape: (506, 13) y Shape: (506, 1)


Reading data from csv file and converting data from frameformat to numpy array

In [3]:

data = pd.read_csv('boston.csv')
print(type(data))

data = data.to_numpy()
print(type(data),data.shape)



<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'> (506, 14)


Extracting input and output training data into X_train, y_train respectively


In [4]:

X_train_temp = data[0:400,0:-1]
X_train = np.zeros((X_train_temp.shape[0],X_train_temp.shape[1]+1))
X_train[:,0] = np.ones((X_train_temp.shape[0]))
X_train[:,1:] = X_train_temp
print("X_train  ", type(X_train),X_train.shape)

y_train = data[0:400,-1]
print("y_train ",type(y_train),y_train.shape)


X_train   <class 'numpy.ndarray'> (400, 14)
y_train  <class 'numpy.ndarray'> (400,)


Extracting input and output testing data into X_test, y_test respectively

In [5]:

X_test_temp = data[400:506,0:-1]
X_test = np.zeros((X_test_temp.shape[0],X_test_temp.shape[1]+1))
X_test[:,0] = np.ones((X_test_temp.shape[0]))
X_test[:,1:] = X_test_temp
print("X_test ", type(X_test),X_test.shape)

y_test = data[400:506,-1]
print("y_test ",type(y_test),y_test.shape)


X_test  <class 'numpy.ndarray'> (106, 14)
y_test  <class 'numpy.ndarray'> (106,)




#**Implementation Simple and Multiple Linear Regression using Gradient Descent (stream data)**



Applying z-score normalization on training and testing input dataset

In [0]:

from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X_train[:,1:])
X_train[:,1:]=scaler.transform(X_train[:,1:])
X_test[:,1:]=scaler.transform(X_test[:,1:])


Generates stream data containing single data record 

In [0]:

def mySGDGeneratorfun(wsize=1):
    print("In myGeneratorfun:")
    cnt = 0
    for i in range(int(X_train.shape[0])):
      X_chunk = X_train[(cnt*wsize):((cnt*wsize)+wsize)]
      y_chunk = y_train[(cnt*wsize):((cnt*wsize)+wsize)]
      cnt = cnt+1
      yield (X_chunk,y_chunk)
      if cnt >= X_train.shape[0]-1:
          break


Initialize Regression parameter matrix theta randomly between 0 and 1

In [8]:

theta=np.zeros(X_train.shape[1])
theta=np.random.uniform(0,1,size=(X_train.shape[1]))
print("Theta:", type(theta), theta.shape)

alpha = 0.01

Theta: <class 'numpy.ndarray'> (14,)


Implementing gradient descent formula on stream data


$\\
\theta_j := \theta_j - \alpha\frac{\partial J(\theta)}{\partial \theta}$


where 

>$J(\theta) = \frac{1}{2m}\sum_{i=1}^{m}(h_\theta(x^{(i)})-y^{(i)})^2$

So final equation for 
>$\theta_j$ given below

>$\theta_j := \theta_j - \alpha\frac{1}{m}\sum_{i=1}^{m}(h_\theta(x^{(i)})-y^{(i)})$   $x_j^{(i)}$

In [9]:

myGenerator = mySGDGeneratorfun(1)
for (x_new,y_new) in myGenerator:
 for i in range(x_new.shape[0]):
    update=np.zeros(x_new[i].shape[0])
    y_pred=np.dot(x_new[i],theta)
    error=y_pred - y_new[i]
    for j in range(x_new.shape[1]):
        update[j]=error*(x_new[i].T)[j]
    theta = theta - (alpha)*update

    
print("Theta:",theta)
print("Theta Shape:", theta.shape)


In myGeneratorfun:
Theta: [20.13472873  0.03478581 -1.29894479  1.05348377  2.09912509  1.25488987
  2.57948811  1.00251384 -1.67730829 -2.36685412  0.0965377  -0.37201363
 -1.05112661 -4.37737189]
Theta Shape: (14,)


Final Testing


In [10]:
#Testing the model built on test set
predictions=np.dot(X_test, theta)

#Computing mean absolute error and mean squared error
print("MAE:", metrics.mean_absolute_error(y_true=y_test,y_pred=predictions))
print("MSE:", metrics.mean_squared_error(y_true=y_test,y_pred=predictions))

MAE: 6.0987966974873515
MSE: 51.6980067365876
